# CatBoost Algorithm

The CatBoost algorithm is based on Gradient Descent which is a powerful technique for classification and regression problems in Machine Learning. In this article, I will introduce you to the CatBoost algorithm in Machine Learning and its implementation using Python.

#### CatBoost Algorithm in Machine Learning
The CatBoost algorithm is based on Gradient Descent and is a powerful technique for supervised machine learning tasks. It will be well suited to problems that involve categorical data. It is widely used for regression and classification tasks and it is also one of the most used algorithms in Kaggle competitions.

The CatBoost algorithm is based on gradient decision trees and when training this model a set of decision trees is built consecutively. As training progresses, each successive tree is built with a reduced loss compared to the previous tree.

In the section below, I will take you through how to implement the CatBoost algorithm in Machine Learning by using the Python programming language.

## CatBoost Algorithm using Python
Now let’s see how to implement the CatBoost algorithm in Machine Learning using Python. Here I will be using the classic Titanic dataset which is one of the most famous datasets in the data science community. Now let’s start by importing the necessary Python libraries and the dataset:

In [1]:
import pandas as pd
from pandas.api.types import is_numeric_dtype
from sklearn.model_selection import train_test_split
import catboost as cb
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
data = pd.read_csv('https://gist.githubusercontent.com/amankharwal/f4c03b6e26c80d8b526f8f44ce717840/raw/fa71405126017e6a37bea592440b4bee94bf7b9e/titanic.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#### Data Preparation
Before training the model, it is very important to prepare the data from the machine learning model, so here I will perform the below mentioned steps for data preparation:

I will first remove the Survived column as that will be the target variable.
Then I will split the data by creating two DataFrames like x and y, one will contain the target variable and the other will contain the useful features for the model.
Next, I’ll convert the ‘Pclass’ column to a string data type, then fill in the null values present in the features.

In [2]:
data.dropna(subset=['Survived'],inplace=True)
X = data[['Pclass', 'Sex', 'Age', 'Fare']]
y = data['Survived']
X['Pclass'] = X['Pclass'].astype('str')
X['Fare'].fillna(0,inplace=True)
X['Age'].fillna(0,inplace=True)

C:\Users\abhij\AppData\Local\Temp\ipykernel_10452\1554941796.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Pclass'] = X['Pclass'].astype('str')
C:\Users\abhij\AppData\Local\Temp\ipykernel_10452\1554941796.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Fare'].fillna(0,inplace=True)
C:\Users\abhij\AppData\Local\Temp\ipykernel_10452\1554941796.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

So, as mentioned at the beginning, the CatBoost algorithm is a powerful machine learning algorithm for categorical features, here I will create two helper functions to generate a list of column indices containing the categorical data, then we need to convert all columns to category data type:

In [3]:
def get_categorical_indicies(X):
    cats = []
    for col in X.columns:
        if is_numeric_dtype(X[col]):
            pass
        else:
            cats.append(col)
    cat_indicies = []
    for col in cats:
        cat_indicies.append(X.columns.get_loc(col))
    return cat_indicies
categorical_indicies = get_categorical_indicies(X)

def convert_cats(X):
    cats = []
    for col in X.columns:
        if is_numeric_dtype(X[col]):
            pass
        else:
            cats.append(col)
    cat_indicies = []
    for col in cats:
        X[col] = X[col].astype('category')
convert_cats(X)

C:\Users\abhij\AppData\Local\Temp\ipykernel_10452\121970121.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype('category')
C:\Users\abhij\AppData\Local\Temp\ipykernel_10452\121970121.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype('category')


Now I’m going to split the data into 20% testing and 80% training:

In [4]:
X_train,X_test,y_train,y_test = train_test_split(X, 
                                                 y, 
                                                 test_size=0.2, 
                                                 random_state=101, 
                                                 stratify=y)

Now before training the model let’s have a look at whether the data is properly balanced or not:

In [5]:
print('Test Survival Rate:',y_test.sum()/y_test.count())

Test Survival Rate: 0.3854748603351955


So we can see that the data is not balanced, there are so many ways to deal with it but I’m just going to downsample the data:

In [6]:
train_df = pd.concat([X,y],axis=1)
survived = train_df[train_df['Survived']==1]
deceased = train_df[train_df['Survived']==0]
deceased = deceased.sample(n=len(survived), random_state=101)
train_df = pd.concat([survived,deceased],axis=0)
X_train = train_df.drop('Survived',axis=1)
y_train = train_df['Survived']

#### Final Step: Training Model
Now let’s train the model by using the CatBoost Algorithm using Python and print the classification report:

In [7]:
train_dataset = cb.Pool(X_train,y_train, 
                        cat_features=categorical_indicies)                                                      
test_dataset = cb.Pool(X_test,y_test,           
                       cat_features=categorical_indicies)
                       
  
model = cb.CatBoostClassifier(loss_function='Logloss',  
                              eval_metric='Accuracy')
grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5,],
        'iterations': [50, 100, 150]}
        
 
model.grid_search(grid,train_dataset)
pred = model.predict(X_test)
print(classification_report(y_test, pred))

0:	learn: 0.7733090	test: 0.7226277	best: 0.7226277 (0)	total: 160ms	remaining: 7.86s
1:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 170ms	remaining: 4.08s
2:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 180ms	remaining: 2.83s
3:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 192ms	remaining: 2.21s
4:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 199ms	remaining: 1.79s
5:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 210ms	remaining: 1.54s
6:	learn: 0.7641682	test: 0.7810219	best: 0.7810219 (1)	total: 219ms	remaining: 1.35s
7:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 228ms	remaining: 1.2s
8:	learn: 0.7641682	test: 0.7810219	best: 0.7810219 (1)	total: 239ms	remaining: 1.09s
9:	learn: 0.7641682	test: 0.7810219	best: 0.7810219 (1)	total: 251ms	remaining: 1s
10:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 263ms	remaining: 932ms
11:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	

1:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 27.4ms	remaining: 658ms
2:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 36.1ms	remaining: 565ms
3:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 44.9ms	remaining: 517ms
4:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 60.7ms	remaining: 546ms
5:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 73ms	remaining: 535ms
6:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 86.3ms	remaining: 530ms
7:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 93.9ms	remaining: 493ms
8:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 108ms	remaining: 491ms
9:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 120ms	remaining: 479ms
10:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 133ms	remaining: 472ms
11:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 141ms	remaining: 447ms
12:	learn: 0.7605119	test: 0.7810219	best: 0.78

3:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 42.9ms	remaining: 493ms
4:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 52.3ms	remaining: 471ms
5:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 60.5ms	remaining: 444ms
6:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 67.4ms	remaining: 414ms
7:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 81.3ms	remaining: 427ms
8:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 95.5ms	remaining: 435ms
9:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 108ms	remaining: 431ms
10:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 122ms	remaining: 434ms
11:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 131ms	remaining: 416ms
12:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 141ms	remaining: 402ms
13:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 151ms	remaining: 389ms
14:	learn: 0.7605119	test: 0.7810219	best: 0

11:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 133ms	remaining: 976ms
12:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 147ms	remaining: 984ms
13:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 161ms	remaining: 989ms
14:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 176ms	remaining: 998ms
15:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 190ms	remaining: 995ms
16:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 200ms	remaining: 977ms
17:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 209ms	remaining: 954ms
18:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 219ms	remaining: 935ms
19:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 229ms	remaining: 916ms
20:	learn: 0.7605119	test: 0.7810219	best: 0.7810219 (1)	total: 239ms	remaining: 898ms
21:	learn: 0.7641682	test: 0.7810219	best: 0.7810219 (1)	total: 248ms	remaining: 880ms
22:	learn: 0.7641682	test: 0.7810219	best: 

13:	learn: 0.7733090	test: 0.7591241	best: 0.7810219 (1)	total: 150ms	remaining: 924ms
14:	learn: 0.7769653	test: 0.7591241	best: 0.7810219 (1)	total: 161ms	remaining: 915ms
15:	learn: 0.7769653	test: 0.7664234	best: 0.7810219 (1)	total: 173ms	remaining: 910ms
16:	learn: 0.7897623	test: 0.7372263	best: 0.7810219 (1)	total: 183ms	remaining: 895ms
17:	learn: 0.7970750	test: 0.7445255	best: 0.7810219 (1)	total: 193ms	remaining: 878ms
18:	learn: 0.7989031	test: 0.7445255	best: 0.7810219 (1)	total: 202ms	remaining: 863ms
19:	learn: 0.8025594	test: 0.7445255	best: 0.7810219 (1)	total: 212ms	remaining: 848ms
20:	learn: 0.8007313	test: 0.7445255	best: 0.7810219 (1)	total: 223ms	remaining: 839ms
21:	learn: 0.7952468	test: 0.7664234	best: 0.7810219 (1)	total: 232ms	remaining: 822ms
22:	learn: 0.8007313	test: 0.7664234	best: 0.7810219 (1)	total: 241ms	remaining: 808ms
23:	learn: 0.8080439	test: 0.7445255	best: 0.7810219 (1)	total: 252ms	remaining: 798ms
24:	learn: 0.8208410	test: 0.7591241	best: 

...

141:	learn: 0.9506399	test: 0.7737226	best: 0.7810219 (1)	total: 3.93s	remaining: 221ms
142:	learn: 0.9506399	test: 0.7737226	best: 0.7810219 (1)	total: 3.97s	remaining: 194ms
143:	learn: 0.9506399	test: 0.7737226	best: 0.7810219 (1)	total: 4.01s	remaining: 167ms
144:	learn: 0.9469835	test: 0.7591241	best: 0.7810219 (1)	total: 4.04s	remaining: 139ms
145:	learn: 0.9469835	test: 0.7591241	best: 0.7810219 (1)	total: 4.07s	remaining: 111ms
146:	learn: 0.9469835	test: 0.7591241	best: 0.7810219 (1)	total: 4.1s	remaining: 83.6ms
147:	learn: 0.9469835	test: 0.7591241	best: 0.7810219 (1)	total: 4.13s	remaining: 55.8ms
148:	learn: 0.9506399	test: 0.7591241	best: 0.7810219 (1)	total: 4.16s	remaining: 27.9ms
149:	learn: 0.9506399	test: 0.7591241	best: 0.7810219 (1)	total: 4.2s	remaining: 0us

bestTest = 0.7810218978
bestIteration = 1

51:	loss: 0.7810219	best: 0.8175182 (36)	total: 1m 18s	remaining: 3.03s
0:	learn: 0.7751371	test: 0.7372263	best: 0.7372263 (0)	total: 16.7ms	remaining: 2.49s
1:	lea

86:	learn: 0.8464351	test: 0.7737226	best: 0.7883212 (33)	total: 1.96s	remaining: 1.42s
87:	learn: 0.8482633	test: 0.7737226	best: 0.7883212 (33)	total: 2s	remaining: 1.41s
88:	learn: 0.8482633	test: 0.7737226	best: 0.7883212 (33)	total: 2.04s	remaining: 1.4s
89:	learn: 0.8482633	test: 0.7737226	best: 0.7883212 (33)	total: 2.08s	remaining: 1.38s
90:	learn: 0.8500914	test: 0.7737226	best: 0.7883212 (33)	total: 2.1s	remaining: 1.36s
91:	learn: 0.8519196	test: 0.7737226	best: 0.7883212 (33)	total: 2.11s	remaining: 1.33s
92:	learn: 0.8519196	test: 0.7737226	best: 0.7883212 (33)	total: 2.14s	remaining: 1.31s
93:	learn: 0.8519196	test: 0.7737226	best: 0.7883212 (33)	total: 2.15s	remaining: 1.28s
94:	learn: 0.8537477	test: 0.7737226	best: 0.7883212 (33)	total: 2.19s	remaining: 1.27s
95:	learn: 0.8500914	test: 0.7737226	best: 0.7883212 (33)	total: 2.2s	remaining: 1.24s
96:	learn: 0.8519196	test: 0.7737226	best: 0.7883212 (33)	total: 2.24s	remaining: 1.22s
97:	learn: 0.8537477	test: 0.7737226	b

33:	learn: 0.8427788	test: 0.7810219	best: 0.7810219 (1)	total: 702ms	remaining: 2.4s
34:	learn: 0.8427788	test: 0.7810219	best: 0.7810219 (1)	total: 740ms	remaining: 2.43s
35:	learn: 0.8446069	test: 0.7810219	best: 0.7810219 (1)	total: 756ms	remaining: 2.4s
36:	learn: 0.8446069	test: 0.7810219	best: 0.7810219 (1)	total: 763ms	remaining: 2.33s
37:	learn: 0.8427788	test: 0.7810219	best: 0.7810219 (1)	total: 797ms	remaining: 2.35s
38:	learn: 0.8464351	test: 0.7737226	best: 0.7810219 (1)	total: 818ms	remaining: 2.33s
39:	learn: 0.8464351	test: 0.7664234	best: 0.7810219 (1)	total: 830ms	remaining: 2.28s
40:	learn: 0.8555759	test: 0.7664234	best: 0.7810219 (1)	total: 870ms	remaining: 2.31s
41:	learn: 0.8555759	test: 0.7737226	best: 0.7810219 (1)	total: 909ms	remaining: 2.34s
42:	learn: 0.8574040	test: 0.7591241	best: 0.7810219 (1)	total: 951ms	remaining: 2.37s
43:	learn: 0.8574040	test: 0.7591241	best: 0.7810219 (1)	total: 965ms	remaining: 2.32s
44:	learn: 0.8574040	test: 0.7664234	best: 0.

132:	learn: 0.9250457	test: 0.7372263	best: 0.7810219 (1)	total: 3.63s	remaining: 464ms
133:	learn: 0.9268739	test: 0.7372263	best: 0.7810219 (1)	total: 3.68s	remaining: 440ms
134:	learn: 0.9287020	test: 0.7372263	best: 0.7810219 (1)	total: 3.72s	remaining: 413ms
135:	learn: 0.9287020	test: 0.7372263	best: 0.7810219 (1)	total: 3.75s	remaining: 387ms
136:	learn: 0.9287020	test: 0.7372263	best: 0.7810219 (1)	total: 3.79s	remaining: 360ms
137:	learn: 0.9287020	test: 0.7372263	best: 0.7810219 (1)	total: 3.83s	remaining: 333ms
138:	learn: 0.9305302	test: 0.7372263	best: 0.7810219 (1)	total: 3.86s	remaining: 306ms
139:	learn: 0.9305302	test: 0.7372263	best: 0.7810219 (1)	total: 3.91s	remaining: 279ms
140:	learn: 0.9305302	test: 0.7372263	best: 0.7810219 (1)	total: 3.95s	remaining: 252ms
141:	learn: 0.9341865	test: 0.7372263	best: 0.7810219 (1)	total: 3.98s	remaining: 225ms
142:	learn: 0.9360146	test: 0.7372263	best: 0.7810219 (1)	total: 4.02s	remaining: 197ms
143:	learn: 0.9360146	test: 0.73

24:	learn: 0.7850877	test: 0.7236842	best: 0.7324561 (2)	total: 578ms	remaining: 578ms
25:	learn: 0.7850877	test: 0.7236842	best: 0.7324561 (2)	total: 593ms	remaining: 547ms
26:	learn: 0.7872807	test: 0.7236842	best: 0.7324561 (2)	total: 629ms	remaining: 536ms
27:	learn: 0.7872807	test: 0.7236842	best: 0.7324561 (2)	total: 646ms	remaining: 507ms
28:	learn: 0.7872807	test: 0.7236842	best: 0.7324561 (2)	total: 654ms	remaining: 473ms
29:	learn: 0.7872807	test: 0.7236842	best: 0.7324561 (2)	total: 663ms	remaining: 442ms
30:	learn: 0.7894737	test: 0.7236842	best: 0.7324561 (2)	total: 694ms	remaining: 425ms
31:	learn: 0.7894737	test: 0.7236842	best: 0.7324561 (2)	total: 714ms	remaining: 402ms
32:	learn: 0.7894737	test: 0.7236842	best: 0.7324561 (2)	total: 750ms	remaining: 386ms
33:	learn: 0.7894737	test: 0.7236842	best: 0.7324561 (2)	total: 761ms	remaining: 358ms
34:	learn: 0.7916667	test: 0.7236842	best: 0.7324561 (2)	total: 802ms	remaining: 344ms
35:	learn: 0.7916667	test: 0.7236842	best: 

So this is how we can use the Catboost Algorithm in Machine Learning using Python. I hope you liked this article on CatBoost Algorithm in Machine Learning and its implementation using Python. Feel free to ask your valuable questions in the comments section below.